In [1]:
from significance_analysis import *
from pymer4.models import Lmer
import bambi
import pandas as pd

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
data = pd.read_parquet("significance_analysis_example/datasets/priorband_data.parquet")

In [3]:
lmem=Lmer("value ~ algorithm + (1|benchmark)",data)
lmem.fit(factors={"algorithm":list(data["algorithm"].unique())})
print(lmem.summary())
print(lmem.post_hoc(marginal_vars=["algorithm"]))#,grouping_vars=["benchmark"])[0])

c:\Users\lenak\OFFLINE_FILES\Anton\Git\lmem-significance\.venv_3_10_0\lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(


Linear mixed model fit by REML [’lmerMod’]
Formula: value~algorithm+(1|benchmark)

Family: gaussian	 Inference: parametric

Number of observations: 86400	 Groups: {'benchmark': 12.0}

Log-likelihood: -244294.170 	 AIC: 488598.341

Random effects:

                  Name     Var    Std
benchmark  (Intercept)  14.804  3.848
Residual                16.707  4.087

No random effect correlations specified

Fixed effects:

Linear mixed model fit by REML [’lmerMod’]
Formula: value~algorithm+(1|benchmark)

Family: gaussian	 Inference: parametric

Number of observations: 86400	 Groups: {'benchmark': 12.0}

Log-likelihood: -244294.170 	 AIC: 488598.341

Random effects:

                  Name     Var    Std
benchmark  (Intercept)  14.804  3.848
Residual                16.707  4.087

No random effect correlations specified

Fixed effects:

             Estimate  2.5_ci  97.5_ci     SE         DF  T-stat  P-val  Sig
(Intercept)     2.557   0.379    4.734  1.111     11.008   2.301  0.042    *
algori

In [4]:
bamb=bambi.Model("value ~ algorithm + (1|benchmark)",data)
print(bamb)
results=bamb.fit()
print(results)


       Formula: value ~ algorithm + (1|benchmark)
        Family: gaussian
          Link: mu = identity
  Observations: 86400
        Priors: 
    target = mu
        Common-level effects
            Intercept ~ Normal(mu: 2.3279, sigma: 19.4629)
            algorithm ~ Normal(mu: [0. 0.], sigma: [29.1943 29.1943])
        
        Group-level effects
            1|benchmark ~ Normal(mu: 0.0, sigma: HalfNormal(sigma: 19.4629))
        
        Auxiliary parameters
            sigma ~ HalfStudentT(nu: 4.0, sigma: 5.5049)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, Intercept, algorithm, 1|benchmark_sigma, 1|benchmark_offset]


Output()